### Instalando e importando as bibliotecas necessárias

In [1]:
!pip install -rq 'requirements.txt'

In [2]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

### Criando o ambiente e extraindo o número de inputs

In [3]:
env = gym.make('CartPole-v0')
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print('Número de inputs: {}'.format(nb_actions))

Número de inputs: 2


### Testando o ambiente com inputs randômicos

In [4]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episódio: {} Score: {}'.format(episode, score))
env.close()

Episódio: 1 Score: 26.0
Episódio: 2 Score: 16.0
Episódio: 3 Score: 32.0
Episódio: 4 Score: 28.0
Episódio: 5 Score: 14.0
Episódio: 6 Score: 29.0
Episódio: 7 Score: 45.0
Episódio: 8 Score: 16.0
Episódio: 9 Score: 17.0
Episódio: 10 Score: 23.0


### Construindo da model

In [5]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = nb_actions, activation = 'linear'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 11,042
Trainable params: 11,042
Non-trainable params: 0
_________________________________________________________________
None


### Configurando e compilando o Agent

In [6]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=1000)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-4), metrics=['mae'])

In [7]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2, nb_max_start_steps=750)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/rl/core.py:153: UserWarning: Env ended before 510 random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.
  warnings.warn(f'Env ended before {nb_random_start_steps} random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.')
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversamp

   15/10000: episode: 1, duration: 0.596s, episode steps:  15, steps per second:  25, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.400 [0.000, 1.000],  loss: 0.550408, mae: 0.570810, mean_q: 0.126588, mean_eps: 0.988750
   29/10000: episode: 2, duration: 0.128s, episode steps:  14, steps per second: 110, episode reward: 14.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.643 [0.000, 1.000],  loss: 0.509433, mae: 0.564630, mean_q: 0.148011, mean_eps: 0.980650


/usr/local/lib/python3.7/dist-packages/rl/core.py:153: UserWarning: Env ended before 441 random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.
  warnings.warn(f'Env ended before {nb_random_start_steps} random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.')
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.7/dist-packages/rl/core.py:153: UserWarning: Env ended before 487 random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.
  warnings.warn(f'Env ended before {nb_random_start_steps} random steps could be performed at the start. You should probably 

   44/10000: episode: 3, duration: 0.139s, episode steps:  15, steps per second: 108, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.467 [0.000, 1.000],  loss: 0.455749, mae: 0.564539, mean_q: 0.203650, mean_eps: 0.967600
   57/10000: episode: 4, duration: 0.106s, episode steps:  13, steps per second: 123, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.692 [0.000, 1.000],  loss: 0.426414, mae: 0.570471, mean_q: 0.243204, mean_eps: 0.955000


...

 9738/10000: episode: 165, duration: 1.535s, episode steps: 200, steps per second: 130, episode reward: 200.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  loss: 6.403872, mae: 25.909695, mean_q: 52.268779, mean_eps: 0.100000


/usr/local/lib/python3.7/dist-packages/rl/core.py:153: UserWarning: Env ended before 176 random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.
  warnings.warn(f'Env ended before {nb_random_start_steps} random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.')


 9938/10000: episode: 166, duration: 1.505s, episode steps: 200, steps per second: 133, episode reward: 200.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.505 [0.000, 1.000],  loss: 6.185717, mae: 26.271432, mean_q: 53.102228, mean_eps: 0.100000


/usr/local/lib/python3.7/dist-packages/rl/core.py:153: UserWarning: Env ended before 631 random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.
  warnings.warn(f'Env ended before {nb_random_start_steps} random steps could be performed at the start. You should probably lower the `nb_max_start_steps` parameter.')


done, took 78.316 seconds


### Avaliação do treinamento em 20 episódios

In [8]:
dqn.test(env, nb_episodes=20, visualize=False)

Testing for 20 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200


### Salvamento dos pesos após a avaliação dos resultados

In [9]:
dqn.save_weights('saves/dqn_weights.h5f', overwrite=False)